In [37]:
#!pip install transformers torch torchdata
#!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached pyarrow-18.0.0.tar.gz (1.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
Using cached datasets-3.1.0-py3-none-any.whl (480 kB)
Using cached multiprocess-0.70.16-py311-none-any.whl (143 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached tqdm-4.67.0-py3-none-any.whl (78 kB)
Using cached xxhash-3.5.0-cp311-cp311-macosx_10_9_x86_64.whl (31 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for pyarrow (pyp

In [38]:
#from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

In [39]:
#Read Dialogue SUmmary Dataset
train_dataset = pd.read_csv("dialogue-sum-dataset/train.csv")
test_dataset = pd.read_csv("dialogue-sum-dataset/test.csv")

In [94]:
model_name='google/flan-t5-large'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [95]:
#TO go to Default Cache directory location
#Finder ---> cmd + Shift + G ---> Paste ~/.cache/huggingface/hub

Cache directory: google/flan-t5-large


In [60]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i in example_indices:
    print(dash_line)
    print('Example ', i)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(test_dataset.iloc[i]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(test_dataset.iloc[i]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  40
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Ex

In [76]:
dialogue = test_dataset.iloc[40]['dialogue']
print("Dialogue: ",dialogue)
print("\n")

summary = test_dataset.iloc[40]['summary']
print("summary: ", summary)
print("\n")

inputs = tokenizer(dialogue, return_tensors='pt')
print("Tokenized Input: ", inputs)
print("\n")

encoded_output = model.generate(inputs["input_ids"], max_new_tokens=50)
print("Encoded Output: ", encoded_output)
print("\n")

output = tokenizer.decode(encoded_output[0], skip_special_tokens=True)
print("Decoded Output: ", output)

Dialogue:  #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.


summary:  #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


Tokenized Input:  {'input_ids': tensor([[ 1713,   345, 13515,   536,  4663,    10,   363,    97,    19,    34,
             6,  3059,    58,  1713,   345, 13515,   357,  4663,    10,  1142,
             3,     9,  1962,     5,    94,    31,     7,     3,   324,    12,
          4169,    57,    82,  1605,     5,  1713,   345, 13515,   536,  4663,
            10,    27,     7,    34,    58,    27,   141,   150,   800,    34,
            47,    78,  1480,     5,    27,   398,    36,   326,   230,     5,
          1713,   345, 1

In [58]:
for i in example_indices:
    dialogue = test_dataset.iloc[i]['dialogue']
    summary = test_dataset.iloc[i]['summary']
    
    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(model.generate(inputs["input_ids"], max_new_tokens=50,)[0], skip_special_tokens=True)
    
    print(dash_line)
    print('Example ', i)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  40
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

------------------------------

In [80]:
##Zero Shot Inference with an Instruction Prompt
for i in example_indices:
    dialogue = test_dataset.iloc[i]['dialogue']
    summary = test_dataset.iloc[i]['summary']

    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Example ', i)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)    
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  40
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

What is the issue the person is facing in the below conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZER

In [81]:
##One Shot Inference
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = test_dataset.iloc[index]['dialogue']
        summary = test_dataset.iloc[index]['summary']
        
        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""
    
    dialogue = test_dataset.iloc[example_index_to_summarize]['dialogue']
    
    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
        
    return prompt

In [82]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also ne

In [83]:
summary = test_dataset.iloc[example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


In [87]:
###Few Shot Inference
example_indices_full = [50, 90, 220]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: Yeah. Just pull on this strip. Then peel off the back.
#Person2#: You might make a few enemies this way.
#Person1#: If they don't think this is fun, they're not meant to be our friends.
#Person2#: You mean your friends. I think it's cruel.
#Person1#: Yeah. But it's fun. Look at those two ugly old ladies. . . or are they men?
#Person2#: Hurry! Get a shot!. . . Hand it over!
#Person1#: I knew you'd come around. . .

What was going on?
#Person1# is about to make a prank. #Person2# thinks it's cruel at first but then joins.



Dialogue:

#Person1#: What's wrong with you, Mr. Polly?
#Person2#: What's wrong? I want a break from this horrible job.
#Person1#: Then, buy a bottle of soft drink.
#Person2#: Would you like to buy a bottle for me in the shop?
#Person1#: It's a problem, because my boss is in that shop now.
#Person2#: Ok, I will go there myself.
#Person1#: Sorry, Mr. Polly.
#Person2#: It doesn't matter. Oh, God, I have only four dollars in my wallet. Is that pos

In [88]:
summary = test_dataset.iloc[example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(inputs["input_ids"],max_new_tokens=50,)[0], 
    skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 recommends upgrading the system and hardware.


In [ ]:
##Generative Configuration Parameters for Inference

In [93]:
#generation_config = GenerationConfig(max_new_tokens=50)
#generation_config = GenerationConfig(max_new_tokens=10)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(inputs["input_ids"], generation_config=generation_config,)[0], 
    skip_special_tokens=True)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 recommends upgrading the system and hardware.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.



In [97]:
generation_config = GenerationConfig(max_new_tokens=50)
#generation_config = GenerationConfig(max_new_tokens=10)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(inputs["input_ids"], generation_config=generation_config,)[0], 
    skip_special_tokens=True)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
Person2 is considering upgrading her system. She would like to add a painting program to her software. She needs a faster processor, a more powerful hard disc, more memory and a faster modem. She doesn't have 
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.



In [32]:
#Sentiment Analysis Sample
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)       # Show all rows
pd.set_option('display.max_colwidth', None)   # Display full column content

# Load tokenizer and model using AutoModel
model_name = "google/flan-t5-base"  # or other Flan-T5 variants
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Function to perform sentiment analysis
def analyze_sentiment(text):
    # Prepare input prompt for Flan-T5
    input_text = f"Classify the sentiment of this text: {text}"
    
    # Tokenize and encode the input text
    inputs = tokenizer(input_text, return_tensors="pt")
    
    # Generate output
    outputs = model.generate(**inputs, max_length=50)
    
    # Decode and return the result
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# Example usage
text = "I love using Vertex AI! It's incredibly powerful and easy to use."
sentiment = analyze_sentiment(text)
print(f"Sentiment: {sentiment}")

#load Sentiment Analysis Dataset
df = pd.read_csv("sentiment-analysis.csv")

df=df.dropna()
newdf=pd.DataFrame(columns = ['Text', 'Sentiment', 'Source','Date/Time', 'User ID', 'Location', 'Confidence Score'])
for i in df.index:
    aux = df.iloc[i,0].split(",")
    newdf.loc[len(newdf)] = aux

newdf['Text'] = newdf['Text'].str.replace('"', '')

newdf['flan-t5-pred'] = [analyze_sentiment(text) for text in newdf["Text"]]
pd.crosstab(newdf["Sentiment"],newdf["flan-t5-pred"])